In [120]:
import pandas as pd
import json
from geopy import distance as dis
from datetime import datetime,timedelta
import math

DIR = "../data/"

files = [
    DIR + "part-00000-5697c6a1-9765-42cf-8ecc-90475faca338-c000.csv" ,
    DIR + "part-00001-5697c6a1-9765-42cf-8ecc-90475faca338-c000.csv" ,
    DIR + "part-00002-5697c6a1-9765-42cf-8ecc-90475faca338-c000.csv" ,
]
file_kakou = DIR + 'kakou_transformed.csv'

In [121]:
kakou_df = pd.read_csv(file_kakou)

## 筛一批数据掉，只选择二环内的， 缩小范围i
kakou_df = kakou_df[kakou_df['location'] == '二环内' ]
print('筛选后卡口数量',len(kakou_df))
available_kakous = list(kakou_df['I_ID'])

筛选后卡口数量 1896


In [122]:
kakou_df.head()

,I_ID,DIRECTION_INDEX,C_NAME,longitude,latitude,location
0,20b5e1cf8694af7a3c1ba4a87f073021,2.0,裕华路站前街西向东,114.479939,38.034575,二环内
1,f4dd765c12f2ef67f98f3558c282a9cd,1.0,南二环富强大街东向西,114.516093,38.000833,二环内
2,d8700cbd38cc9f30cecb34f0c195b137,1.0,南二环汇祥路桥上东向西,114.483926,38.002373,二环内
3,b4d168b48157c623fbd095b4a565b5bb,1.0,北二环沿东街东向西,114.514628,38.078750,二环内
4,4a08142c38dbe374195d41c04562d9f8,2.0,北二环中华大街西向东,114.477580,38.082932,二环内


In [123]:
## 看两个卡口是否靠近
## https://geopy.readthedocs.io/en/latest/#module-geopy.distance
def isTwoKakousNearBy(lon_a ,lat_a ,lon_b ,lat_b):
    th = 1  # 1000米
    a = ( lat_a , lon_a )
    b = ( lat_b , lon_b )
#     distance = dis.distance(a , b).km       
    distance = dis.great_circle(a , b).km  #快一些 （大圆模型）
    return distance <= th

## 构造邻近卡口 Map
# near_kakou_map = {}
# for i in range(0,len(kakou_df)):
#     print(i)
#     k_a = kakou_df.iloc[i]
#     if(k_a['I_ID'] in near_kakou_map.keys()):
#         continue
#     near_kakous = set()
#     for j in range(0,len(kakou_df)):
#         k_b = kakou_df.iloc[j]
#         if(isTwoKakousNearBy( k_a['longitude'],k_a['latitude'],k_b['longitude'],k_b['latitude']) ):
#             near_kakous.add(k_b['I_ID'])
#     near_kakous = list(near_kakous)
#     if(len(near_kakous) > 0):
#         near_kakou_map[k_a['I_ID']] = near_kakous

with open(DIR + 'near_kakou_map.json', 'r') as f:
    near_kakou_map = json.load(f)

In [124]:
# with open(DIR + 'near_kakou_map.json', 'w') as outfile:
#     json.dump( near_kakou_map , outfile )

In [125]:
traj_0_df = pd.read_csv(files[0])
# traj_1_df = pd.read_csv(files[1])
# traj_2_df = pd.read_csv(files[2])

# traj_df = pd.concat([traj_0_df,traj_1_df,traj_2_df])  # 合并三个
# print(len(traj_0_df) , len(traj_1_df) , len(traj_2_df))
# print(len(traj_df))
traj_0_df = traj_0_df[0:1000000]
traj_df = traj_0_df

In [126]:
# print("最早时间", traj_df['PASS_TIME'].min())
# print("最晚时间", traj_df['PASS_TIME'].max())

In [127]:
## 按照车牌分组 
## 并按时间排序
def sortTime(row):
    row = row.sort_values(by="PASS_TIME")
    return row

trajs_df =  traj_df.groupby('PLATE_NO' ).apply(sortTime)

In [128]:
trajs_df = trajs_df.reset_index(drop=True) ## 重新设置index为0开始

In [129]:
def dateTime2TimeSlot(dt_str):
#     TIME_GAP = 10  # 10分钟为一个时间间隔
    TIME_GAP = 30  # 30分钟为一个时间间隔
#     dt_str = '2019-01-03 00:17:05.963000' 
    time = dt_str.split('.')[0] #去掉小数点
    dt = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
    day_start = dt.strftime('%Y-%m-%d') + ' 00:00:00'  # 这天零点
    dt_start = datetime.strptime(day_start, '%Y-%m-%d %H:%M:%S')
    delta = dt - dt_start  # 差
    timeSlot = math.ceil(delta.seconds / (60 * TIME_GAP))
    return int(timeSlot)

In [130]:
T = []
for t in range(0,144):
    T.append({})

    
## 轨迹处理
trajs_df_filter = pd.DataFrame()
length = len(trajs_df)
print(length)
for i in range(1,length-1):
    cur   = trajs_df.iloc[i]
    last  = trajs_df.iloc[i-1]
    
    if(i % 10000 == 0):
        print(i)
        
    ## 是否是一辆车的轨迹 
    if( cur['PLATE_NO'] != last['PLATE_NO'] ):
        continue
    last_cross = last['CROSSING_ID']
    cur_cross  = cur['CROSSING_ID']

    ## 看是两次否处于同一个点
    if(last_cross == cur_cross):
        continue
    
    ## 是否处于范围内
    if((last_cross not in available_kakous) or (cur_cross not in available_kakous)):
        continue
    
    ## 两个点是否相邻 （位置相近）
    if((last_cross in near_kakou_map.keys()) and (cur_cross in near_kakou_map[last_cross])):
#         trajs_df_filter = trajs_df_filter.append({
#             'plate'      : cur['PLATE_NO'],
#             'startKakou' : last_cross,
#             'endKakou'   : cur_cross,
#             'timePeriod' : dateTime2TimeSlot(cur['PASS_TIME'])
#         },ignore_index=True)
        t = int(dateTime2TimeSlot(cur['PASS_TIME']))
        p = cur['PLATE_NO']
        T[t][p] = {
            's' : last_cross,
            'e' : cur_cross
        }
# T

1000000
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000


In [131]:
# len(T[1].keys())
for i in range(0,20):
#     print(len(T[i].keys()))
    if(len(T[i].keys()) > 0):
        print(i,len(T[i].keys()))
        T_df_i = pd.DataFrame.from_dict(T[i] , orient='index')
        T_df_i.to_csv(DIR + 't_df_' + str(i) +'.csv' , index=False )

1 8540
2 7039
3 5995
4 5090
5 3898
6 3462
7 3353
8 3245
9 3128
10 3458
11 4517
12 1627
13 4360
14 132


In [132]:
# trajs_df_filter.to_csv('../data/trajs_forvis_.csv' , index=False )
# # len(trajs_df_filter)